In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)


In [2]:
# ---Set project root + locate CSV---
from pathlib import Path

# Notebook is inside /notebooks, so project root is one level up
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data" / "raw"

# List what files are in data/raw to confirm
files = list(DATA_DIR.glob("*"))
files

[WindowsPath('c:/Users/shahz/OneDrive/Desktop/Folders/Clinical-Risk-Prediciton/data/raw/diabetic_data.csv')]

In [3]:
# ---Load Dataset---
csv_path = DATA_DIR / "diabetic_data.csv"
if not csv_path.exists():
    # Fallback: load the first CSV found in data/raw
    csvs = list(DATA_DIR.glob("*.csv"))
    if len(csvs) == 0:
        raise FileNotFoundError(f"No CSV files found in: {DATA_DIR}")
    csv_path = csvs[0]

df = pd.read_csv(csv_path)
df.shape

(101766, 50)

In [ ]:
# ---Quick look---
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
# ---Replace “?” with NaN + basic missingness---
df = df.replace("?", np.nan)

missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
missing_pct.head(20)

weight                      96.858479
max_glu_serum               94.746772
A1Cresult                   83.277322
medical_specialty           49.082208
payer_code                  39.557416
race                         2.233555
diag_3                       1.398306
diag_2                       0.351787
diag_1                       0.020636
patient_nbr                  0.000000
time_in_hospital             0.000000
admission_source_id          0.000000
num_lab_procedures           0.000000
encounter_id                 0.000000
admission_type_id            0.000000
discharge_disposition_id     0.000000
gender                       0.000000
age                          0.000000
number_inpatient             0.000000
number_emergency             0.000000
dtype: float64

In [ ]:
# ---Target creation (Readmitted within 30 days)---
df["readmitted"].value_counts(dropna=False)

readmitted
NO     54864
>30    35545
<30    11357
Name: count, dtype: int64

In [7]:
df["target_readmit_30"] = (df["readmitted"] == "<30").astype(int)
df["target_readmit_30"].value_counts(normalize=True)


target_readmit_30
0    0.888401
1    0.111599
Name: proportion, dtype: float64

In [ ]:
# ---Basic sanity checks + column overview---
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 51 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In [9]:
df.describe(include="all").T.head(20)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
encounter_id,101766.0,NaN,NaN,NaN,165201645.622978,102640295.983457,12522.0,84961194.0,152388987.0,230270887.5,443867222.0
patient_nbr,101766.0,NaN,NaN,NaN,54330400.694947,38696359.346534,135.0,23413221.0,45505143.0,87545949.75,189502619.0
race,99493,5,Caucasian,76099,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gender,101766,3,Female,54708,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,101766,10,[70-80),26068,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weight,3197,9,[75-100),1336,NaN,NaN,NaN,NaN,NaN,NaN,NaN
admission_type_id,101766.0,NaN,NaN,NaN,2.024006,1.445403,1.0,1.0,1.0,3.0,8.0
discharge_disposition_id,101766.0,NaN,NaN,NaN,3.715642,5.280166,1.0,1.0,1.0,4.0,28.0
admission_source_id,101766.0,NaN,NaN,NaN,5.754437,4.064081,1.0,1.0,7.0,7.0,25.0
time_in_hospital,101766.0,NaN,NaN,NaN,4.395987,2.985108,1.0,2.0,4.0,6.0,14.0


In [ ]:
# ---Define columns to drop (avoid leakage / IDs)---
DROP_COLS = ["readmitted", "encounter_id"]  # drop label + pure ID
TARGET_COL = "target_readmit_30"
GROUP_COL = "patient_nbr"  # used for split only

In [ ]:
# ---Patient-level train/test split (no patient overlap)---
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(df, groups=df[GROUP_COL]))

train_df = df.iloc[train_idx].copy()
test_df  = df.iloc[test_idx].copy()

(train_df.shape, test_df.shape)

((81613, 51), (20153, 51))

In [12]:
# Confirm zero overlap in patient IDs
overlap = set(train_df[GROUP_COL]).intersection(set(test_df[GROUP_COL]))
len(overlap)


0

In [ ]:
# ---Build X/y (keep patient_nbr out of features)---
y_train = train_df[TARGET_COL]
y_test  = test_df[TARGET_COL]

X_train = train_df.drop(columns=DROP_COLS + [TARGET_COL, GROUP_COL])
X_test  = test_df.drop(columns=DROP_COLS + [TARGET_COL, GROUP_COL])

X_train.shape, X_test.shape


((81613, 47), (20153, 47))

In [ ]:
# ---Preprocessing + baseline model pipeline (Logistic Regression)---
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()
num_cols = [c for c in X_train.columns if c not in cat_cols]

numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop"
)

clf = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

clf


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('prep', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains s

In [ ]:
# ---Train + evaluate (ROC-AUC + PR-AUC)---
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, confusion_matrix


In [ ]:

clf = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LogisticRegression(
        solver="saga",
        max_iter=5000,
        class_weight="balanced",
        random_state=42
    ))
])

clf.fit(X_train, y_train)

proba = clf.predict_proba(X_test)[:, 1]
pred  = (proba >= 0.5).astype(int)

roc = roc_auc_score(y_test, proba)
pr  = average_precision_score(y_test, proba)

print("ROC-AUC:", roc)
print("PR-AUC :", pr)
print("\nConfusion Matrix (threshold=0.5):")
print(confusion_matrix(y_test, pred))
print("\nClassification Report (threshold=0.5):")
print(classification_report(y_test, pred, digits=3))


ROC-AUC: 0.6311630181490759
PR-AUC : 0.1825543751310967

Confusion Matrix (threshold=0.5):
[[11761  6241]
 [ 1007  1144]]

Classification Report (threshold=0.5):
              precision    recall  f1-score   support

           0      0.921     0.653     0.764     18002
           1      0.155     0.532     0.240      2151

    accuracy                          0.640     20153
   macro avg      0.538     0.593     0.502     20153
weighted avg      0.839     0.640     0.708     20153



In [ ]:
# ---Thresholding by alert budget (top-K highest risk)---

alert_frac = 0.10
k = max(1, int(alert_frac * len(proba)))

# Threshold = probability cutoff that flags top-k highest risk
threshold = float(np.sort(proba)[-k])

pred_alerts = (proba >= threshold).astype(int)

print(f"Alert fraction: {alert_frac:.0%}  |  Alerts: {pred_alerts.sum()} / {len(pred_alerts)}")
print(f"Threshold used: {threshold:.6f}")

print("\nConfusion Matrix (top-K threshold):")
print(confusion_matrix(y_test, pred_alerts))

print("\nClassification Report (top-K threshold):")
print(classification_report(y_test, pred_alerts, digits=3))


Alert fraction: 10%  |  Alerts: 2015 / 20153
Threshold used: 0.657941

Confusion Matrix (top-K threshold):
[[16443  1559]
 [ 1695   456]]

Classification Report (top-K threshold):
              precision    recall  f1-score   support

           0      0.907     0.913     0.910     18002
           1      0.226     0.212     0.219      2151

    accuracy                          0.839     20153
   macro avg      0.566     0.563     0.564     20153
weighted avg      0.834     0.839     0.836     20153

